### Data visualizer to explore data ranges and experiment with data values

![Greedybusiness](/home/jeffreymo572/Kaggles/common/images/Monopoly.jpg)

In [1]:
# Importing
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sn

# Dataset
data_dir = "~/Kaggles/data/S3E7/train.csv"
df = pd.read_csv(data_dir)

In [ ]:
# Column info
df.info()

In [9]:
df.head(10)

,id,num_adults,num_children,num_weekend_nights,num_week_nights,meal_plan,parking,room_type,lead_time,year,month,date,market_segment_type,repeated_customer,num_prev_cancellations,num_prev_not_cancelled,avg_price_per_room,num_special_requests,booking_status
0,0,2,0,0,2,1,0,0,9,2018,1,14,1,1,11,0,67.50,0,0
1,1,2,0,1,2,0,0,0,117,2018,7,29,0,0,0,0,72.25,0,0
2,2,2,0,0,1,0,0,0,315,2018,12,2,0,0,0,0,52.00,0,0
3,3,1,0,0,2,1,0,0,32,2018,12,1,1,0,0,0,56.00,0,0
4,4,2,0,1,0,0,0,0,258,2018,10,16,0,0,0,0,100.00,0,1
5,5,0,2,2,5,0,0,2,215,2018,10,6,1,0,0,0,212.06,0,1
6,6,2,0,0,1,2,0,0,320,2018,8,18,0,0,0,0,115.00,1,1
7,7,2,0,0,2,2,0,0,265,2018,6,24,0,0,0,0,87.00,0,0
8,8,2,0,2,3,0,0,0,189,2018,10,8,0,0,0,0,72.75,0,0
9,9,3,0,0,3,0,0,0,137,2018,12,22,1,0,0,0,117.30,2,0


In [2]:
# Renaming columns to be more readable
# Current unknowns: market_segment_type, 
df.columns = ["id", 'num_adults', 'num_children', 'num_weekend_nights', 'num_week_nights', 'meal_plan', 
              'parking', 'room_type', 'lead_time', 'year', 'month', 'date', 'market_segment_type', 'repeated_customer',
              'num_prev_cancellations', 'num_prev_not_cancelled', 'avg_price_per_room', 'num_special_requests',
              'booking_status']

for i, value in enumerate(df.columns.tolist()):
    print(f"{i}: {value}")

0: id
1: num_adults
2: num_children
3: num_weekend_nights
4: num_week_nights
5: meal_plan
6: parking
7: room_type
8: lead_time
9: year
10: month
11: date
12: market_segment_type
13: repeated_customer
14: num_prev_cancellations
15: num_prev_not_cancelled
16: avg_price_per_room
17: num_special_requests
18: booking_status


#### Possible Notes for cleaning
##### Additions
* Insert ratio of cancellations at index 16 
    * `num_prev_cancellations/(num_prev_not_cancelled+avg_price_per_room)`
* Is weekend
    * `pd.Timestamp(f"{year}-{month:.2f}-{day:.2f}").dayofweek`
    * Note: Monday is 0
* Is holiday
    * Might have to hard code: https://en.wikipedia.org/wiki/Federal_holidays_in_the_United_States
* Total people
    * `num_adults+num_children`
##### Removals
* ID
* meal_plan
* date -> weekday & holiday
##### Modifications
* Month/Year/Date into one column?
    * Maybe not since month/year may have seasonal impact on cancellation 

# KNOWN CORRELATIONS
**PEOPLE WHO ORDER MEAL PLAN 3 HAVE A 83% CHANCE TO CANCEL!!!** \
**PEOPLE WITH A BOOKING STATUS OF 0 WILL CANCEL!!!!**

In [8]:
# Information about data 
# Mean, median, mode, variance, std, etc.
df_info = pd.DataFrame()

df_info['var'] = df.var()
df_info['mean'] = df.mean()
df_info['std'] = df.std()
df_info['var/mean'] = df.var()/df.mean()
df_info['std/mean'] = df.std()/df.mean()

df_info.reset_index(inplace=True)
df_info.columns = ['category', 'var', 'mean', 'std', 'var/mean', 'std/mean']

# Dropping and replacing values
df_info = df_info.drop(0)

df_info

,category,var,mean,std,var/mean,std/mean
1,num_adults,0.275573,1.920713,0.524950,0.143474,0.273310
2,num_children,0.202615,0.141093,0.450128,1.436046,3.190303
3,num_weekend_nights,0.784452,0.884632,0.885693,0.886756,1.001200
4,num_week_nights,2.037270,2.398005,1.427330,0.849569,0.595216
5,meal_plan,0.345361,0.239192,0.587674,1.443861,2.456908
6,parking,0.024612,0.025249,0.156884,0.974774,6.213357
7,room_type,0.693232,0.428931,0.832605,1.616184,1.941117
8,lead_time,6572.238347,103.888029,81.069343,63.262711,0.780353
9,year,0.123057,2017.856295,0.350795,0.000061,0.000174
10,month,8.005479,7.593539,2.829395,1.054249,0.372606
